In [26]:
import sys
import json
from pathlib import Path
from dateutil import parser
from math import pi

import requests
from shapely.geometry import shape, Point
import geopandas as gpd
import pandas as pd
import numpy as np


DIR = Path('..')
sys.path.append(str(DIR))

import gtfstk as gt
import pyotp as otp
import arcgdfconvertor as agc
import gtfsanalyst as ga


DATA_DIR = DIR/'data/'


%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
# setting the analysis date and time
dt = parser.parse('11/Jan/2018 8:00:00 AM')
city = 'Vancouver'
od_points_path = str(DATA_DIR/'Cities.gdb'/'{0}_Hex_Polygon'.format(city))
feed_path = str(DATA_DIR/'gtfs feeds'/'{0}_GTFS.zip'.format(city))
od_csv_path = str(DATA_DIR/'ODs'/'{0}_detailed_od.csv'.format(city))

# OD matrix

In [28]:
#Read file
od_matrix = pd.read_csv(od_csv_path)
od_matrix = od_matrix.drop(od_matrix.columns.values[0], axis =1)

#convert durations into minutes
od_matrix['duration'] = od_matrix['duration']/60
od_matrix['waitTime'] = od_matrix['waitTime']/1000/60

# Summarise OD matrix

In [29]:
#make a summary table
od_summary = pd.pivot_table(
    od_matrix, 
    index=['trip_name'], 
    values=['duration', 'distance', 'waitTime'],
    columns=['mode'], 
    aggfunc=[np.sum],
    fill_value=0).reset_index()
od_summary['from']= od_summary.trip_name.apply(lambda x: x.split()[1])
od_summary['to']= od_summary.trip_name.apply(lambda x: x.split()[3])
od_summary['invehicle_time']= (od_summary[('sum', 'duration', 'BUS')]+
                                od_summary[('sum', 'duration', 'FERRY')]+
                                od_summary[('sum', 'duration', 'RAIL')])

od_summary['walk_time']= (od_summary[('sum', 'duration', 'WALK')])

od_summary['waitTime']= (od_summary[('sum', 'waitTime', 'BUS')]+
                         od_summary[('sum', 'waitTime', 'FERRY')]+
                         od_summary[('sum', 'waitTime', 'RAIL')])
od_summary['travel_time']= (od_summary['walk_time']+
                            od_summary['invehicle_time']+
                            od_summary['waitTime'])

# GIS feature class

In [30]:
#load od points
pow_df = agc.fc_to_gdf(od_points_path)

pow_df.columns = pd.MultiIndex.from_product([['sum'], [''], pow_df.columns])

od_summary = od_summary.merge(
    pow_df[[('sum', '', 'Unique_ID'), ('sum', '', 'POW')]], 
    left_on = ['to'],
    right_on = [('sum', '','Unique_ID')],
    how='left').drop(('sum', '','Unique_ID'), axis = 1)

od_summary.head()

trip_name           sum                                        \
                             distance                        duration         
mode                              BUS FERRY RAIL      WALK        BUS FERRY   
0     from 1001 to 1002   4469.154894   0.0  0.0   635.468   6.366667     0   
1     from 1001 to 1003  32663.785500   0.0  0.0   327.763  76.633333     0   
2     from 1001 to 1004  15962.099872   0.0  0.0  1014.383  37.033333     0   
3     from 1001 to 1006  25941.482705   0.0  0.0   702.926  53.633333     0   
4     from 1001 to 1007  24711.078410   0.0  0.0  1228.988  47.416667     0   

                                                         from    to  \
                        waitTime                                      
mode RAIL       WALK         BUS FERRY RAIL        WALK               
0     0.0   8.250000    0.016667   0.0  0.0    0.016667  1001  1002   
1     0.0   4.200000   75.233333   0.0  0.0    0.016667  1001  1003   
2     0.0  12.833333    0.016667   0.0  0.0  108.483333  1001  1004   
3     0.0   9.016667  143.750000   0.0  0.0    0.016667  1001  1006   
4     0.0  16.033333    0.016667   0.0  0.0   65.050000  1001  1007   

     invehicle_time  walk_time    waitTime travel_time          sum  
                                                                     
mode                                                            POW  
0          6.366667   8.250000    0.016667   14.633333  4607.948958  
1         76.633333   4.200000   75.233333  156.066667  3276.004753  
2         37.033333  12.833333    0.016667   49.883333  1530.963181  
3         53.633333   9.016667  143.750000  206.400000  2795.636396  
4         47.416667  16.033333    0.016667   63.466667  1854.797740

# Calculate distance decay

In [31]:
f_names = dict()
for i in range(20, 65, 5):
    f_name = 'dist_decay_{0}'.format(i)
    od_summary[f_name] = od_summary['travel_time'].apply(lambda x: 1/(1+math.exp(0.2*(x-i))) )*od_summary[('sum', '', 'POW')]


for i in range(20, 65, 5):
    f_names['dist_decay_{0}'.format(i)] = 'sum'
    
Hex_dist_decay_summary = od_summary.groupby('from', as_index=False).agg(f_names)
Hex_dist_decay_summary.head()

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\pandas\core\groupby.py:4291: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


,from,dist_decay_20,dist_decay_25,dist_decay_30,dist_decay_35,dist_decay_40,dist_decay_45,dist_decay_50,dist_decay_55,dist_decay_60
0,1001,32541.842758,64943.309285,115078.471948,176035.927506,235613.663946,287804.599623,334837.879684,380540.089688,423558.171142
1,1002,52862.410833,98239.921699,162144.267187,234590.592468,303289.221552,364739.105864,422170.926808,480533.758416,540987.969984
2,1003,42855.025084,84295.916551,147102.001370,223761.406416,299745.710562,367394.934642,427425.521869,484990.282574,543216.312267
3,1004,6900.359513,14145.490259,26808.608241,47597.982652,81018.984702,133367.837168,208857.595138,302301.489777,400323.797203
4,1006,7411.795217,12929.060628,22765.004869,39674.588322,66963.055537,110503.251777,177450.462683,266073.883716,362755.128257


In [32]:
Hex_dist_decay_summary.to_csv(str(DATA_DIR/'OD_Summaries'/'{0}_Hex_dist_decay_summary.csv'.format(city)))